In [7]:
import pandas as pd
import os

In [8]:
def obtain_data():
    file_list = []
    directory = "datasets"
    for entry in os.scandir(directory):
        if entry.path.endswith(".xlsx"):
            file_list.append(entry.path)
    return file_list

file_list = obtain_data()

In [9]:
def format_data(df, s_a_c_df):
    statename = df.iloc[6][1]
    state_code = s_a_c_df.loc[s_a_c_df['State'] == statename].iloc[0][2]
    df = df.rename(df.iloc[10], axis='columns')
    df = df[11:]
    df.reset_index(drop=True)
    df = df.astype(str).astype(float)
    df['Year'] = df['Year'].astype(int)
    df["StateCode"] = state_code
    return df

def read_data(file_list):
    s_a_c_df = pd.read_csv("datasets/state_abbreviation_code.csv")
    main_df = pd.DataFrame(columns=['Year','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec','Annual'])
    for file in file_list:
            temp_df = pd.read_excel(file)
            temp_df = format_data(temp_df, s_a_c_df)
            main_df = main_df.append(temp_df, ignore_index=True)
        
    return main_df

def remove_2020(df):
    df = df[df['Year'] != 2020]
    return df

sta_unem_df = read_data(file_list)
sta_unem_df_no_2020 = remove_2020(sta_unem_df)

In [13]:
import plotly.express as px

def generate_choro_fig(unemployment_df):
    fig = px.choropleth(unemployment_df, locations='StateCode', locationmode='USA-states', color='Annual', range_color=[2,18], animation_frame='Year', projection='albers usa', labels={'Annual':'Unemployment Rate (%)'})
    return fig
generate_choro_fig(sta_unem_df_no_2020)

In [15]:
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html

fig = generate_choro_fig(sta_unem_df_no_2020)

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(children=[
    html.H1(children="Unemployment In The United States"),
    html.Div(children='''Unemployment Rate (%) Of Each State From 1976 To 2019'''),
    dcc.Graph(
        id='Unemployment By State Choropleth',
        figure=fig
    )
])

if __name__ == '__main__':
    app.run_server(mode='inline')